In [ ]:
!pip install langchain
!pip install openai
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchainhub
!pip install tiktoken
!pip install langchain_openai
!pip install --upgrade rank_bm25
!pip install langchain-community langchain-core
!pip install -U langchain langchain-openai

import os
from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo
)
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chains import ConversationChain
from langchain.vectorstores import LanceDB
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents import create_openai_functions_agent
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
import re
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
import os


os.environ['OPENAI_API_KEY'] = ''

import openai


os.environ['OPENAI_API_BASE'] =   'https://api.openai.com/v1'

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector = FAISS.load_local("faiss_index", embedding_function,allow_dangerous_deserialization=True)

retriever = vector.as_retriever(search_kwargs={"k": 15})

llm = ChatOpenAI(model="gpt-4o", temperature=0.02)

retriever_MQ = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

retriever_tool = create_retriever_tool(
    retriever_MQ,
    "literature_search",
    "Search for information about input questions. For any questions about battery, you must use this tool!",
)

tools = [retriever_tool]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''
            You are recognized as a leading authority in the field of battery, tasked with the pivotal role of providing expert responses to inquiries. Your responses must embody the highest standards of accuracy, comprehensiveness, and depth of knowledge. Approach each question with a methodical and thoughtful mindset, ensuring your answers are:

            Informative: Furnish detailed insights, drawing upon the most relevant facts and figures.
            Correct: Ensure factual accuracy in every aspect of your response.
            Knowledgeable: Display a profound understanding of the subject matter, including advanced concepts and recent advancements in the field of battery.
            Holistic: Offer a well-rounded perspective, considering various facets of the topic at hand.
            Simplicity: Make sure to be as concise as possible under the above requirements.
            As you address each question, please adhere to the following guidelines:

            Cite Examples: When referencing data or examples from the provided literature, include comprehensive information to contextualize your points effectively. Clearly indicate these instances by stating, "For example," followed by a detailed explanation.
            Stay On Topic: Concentrate solely on the query posed. Your reply should be closely aligned with the question, avoiding tangential or unrelated content.
            Format Your Answer: Present your response in a structured manner, using either bullet points or numbered lists for clarity and ease of understanding.
            Before responding, take a moment to center yourself. Breathe deeply, and proceed with a step-by-step analytical approach, ensuring that your expertise shines through in a manner that is both engaging and enlightening.

            ''',
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, reduce_k_below_max_tokens=True)

input= " In the context of high-energy all-solid-state lithium batteries, how does the use of Ag–C composite anodes improve lithium deposition uniformity, and what mechanisms contribute to the suppression of dendrite formation during cycling? "

breakdown_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", '''
        You are an expert in battery, and your task is to answer user's question professionally.
        You need to consider conprehensive aspects in battery to reach the final answers, however your answers must only be in material science, focus on the scientific facts and methodology.
        Your job is to search for information in literature review and identify the key aspects that are answers to the input question.
        Make sure you are directly answering the subject of the input question(for example: method, defenition, components, mechanism), use termninogy to describe the apects
        ......
        '''),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


agent_breakdown = create_openai_functions_agent(llm, tools, breakdown_prompt)

agent_breakdown_executor = AgentExecutor(agent=agent_breakdown, tools=tools, verbose=True, reduce_k_below_max_tokens=True)

breakdown_a=agent_breakdown_executor.invoke({"input":input, 'chat_history':[]})

thoughts=breakdown_a['output']

step_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", '''
        You are an expert in battery, and your task is to answer user's question professionally.
        You are given steps toward answering a question on battery
        Informative: Furnish detailed insights, drawing upon the most relevant facts and figures.
        Correct: Ensure factual accuracy in every aspect of your response.
        Knowledgeable: Display a profound understanding of the subject matter, including advanced concepts and recent advancements in the field of battery.
        Holistic: Offer a well-rounded perspective, considering various facets of the topic at hand.
        Simplicity: Make sure to be as concise as possible under the above requirements.
        As you address each question, please adhere to the following guidelines:

        Stay On Topic: Concentrate solely on the query posed. Your reply should be closely aligned with the question, avoiding tangential or unrelated content.
        Before responding, take a moment to center yourself. Breathe deeply, and proceed with a step-by-step analytical approach, ensuring that your expertise shines through in a manner that is both engaging and enlightening.

        '''),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent_step = create_openai_functions_agent(llm, tools, step_prompt)

agent_step_executor = AgentExecutor(agent=agent_step, tools=tools, verbose=True, reduce_k_below_max_tokens=True)

thought=agent_step_executor.invoke({"input": "To address the question "+input+", we will first analyze a variety of aspects laid out for you. "+thoughts+"Your answer should involve professional terminology, make sure every point is factual and correct.", 'chat_history':[]})
thoughts+=thought['output']

chat_history=[HumanMessage(content=thought['input']),AIMessage(content=thought['output'])]

vector2 = FAISS.load_local("faiss_index", embedding_function,allow_dangerous_deserialization=True)

retriever2 = vector.as_retriever(search_kwargs={"k": 10})

llm2 = ChatOpenAI(model="gpt-4o", temperature=0.05)

retriever_MQ2 = MultiQueryRetriever.from_llm(
    retriever=retriever2, llm=llm2
)

retriever_tool2 = create_retriever_tool(
    retriever_MQ2,
    "literature_search",
    "Search for information about input questions. For any questions about battery, you must use this tool!",
)

tools2 = [retriever_tool2]

critique_prompt = ChatPromptTemplate.from_messages(
   [
        ("system", '''

        The task requires a detailed critique of an existing explanation concerning technologies for recycling lithium-ion batteries. This critique is intended to serve as guidance for another individual tasked with revising the original answer. The critique must rigorously address specific points:

        Fact-Checking:
        Scrutinize the original answer for discrepancies or outdated information by cross-verifying each mentioned detail against contemporary research and findings in the lithium-ion battery domain. Highlight inaccuracies and ensure that every fact aligns with the most recent studies and technological advancements.
        Supplement:
        Dive into the existing literature review to identify and incorporate missing crucial information not covered in the original answer.
        Avoiding Hallucinations:
        Identify any baseless assertions, speculative statements, or inaccuracies within the original response. Highlight instances where the presented information lacks direct support from the literature review or recognized sources in the field.
        Accuracy and Precision:
        Evaluate the technical accuracy and precision of the original answer's descriptions of recycling methods. Identify and correct inaccuracies or over-generalizations, particularly regarding process efficiencies, environmental impacts, and the validation of technological advancements through recent studies.
        Instructions for Revision:
        The individual responsible for revising the original answer should address all issues identified in this critique. The revised answer must correct any inaccuracies, align with the latest research, and try your best to ensure that you focus on input when modifying content.
        The revised answer should be factual, precise, and devoid of speculative content, adhering closely to established sources within the field.
        Simplicity: Make sure to be as concise as possible under the above requirements.
        The ultimate goal is for the revised answer to be thoroughly informed by this critique, resulting in a comprehensive and accurate response that meets academic standards for inclusion in a literature review on lithium-ion battery.

        Provide your answer to the questions as well as your revision to the original question.

        '''),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent_critique = create_openai_functions_agent(llm2, tools2, critique_prompt)

agent_critique_executor = AgentExecutor(agent=agent_critique, tools=tools2, verbose=True, reduce_k_below_max_tokens=True)


critique=agent_critique_executor.invoke({"input": "Here's the original question you need to provide insights for: "+input+"You need to provide your answers and revision to the answer in the chat history.", 'chat_history':chat_history})


chat_history=[HumanMessage(content=thought['input']),AIMessage(content=thought['output']),HumanMessage(content=critique['input']),AIMessage(content=critique['output'])]

revise_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", '''
You are an expert in battery, and your task is to answer user's question professionally.
        You are given steps toward answering a question on battery
        Informative: Furnish detailed insights, drawing upon the most relevant facts and figures.
        Correct: Ensure factual accuracy in every aspect of your response.
        Knowledgeable: Display a profound understanding of the subject matter, including advanced concepts and recent advancements in the field of battery.
        Holistic: Offer a well-rounded perspective, considering various facets of the topic at hand.
        As you address each question, please adhere to the following guidelines:
        your task is to answer user's question professionally.
        Given the feedback on your initial answer regarding battery, the goal now is to revise your response to make it more informative, accurate, and research-backed.
        You are now giving the final answer, make sure the final answer is based on facts and literature.
        Stay On Topic: Concentrate solely on the query posed. Your reply should be closely aligned with the question, avoiding tangential or unrelated content.
        Format Your Answer: Present your response in a structured manner, using either bullet points or numbered lists for clarity and ease of understanding.
        Before responding, take a moment to center yourself. Breathe deeply, and proceed with a step-by-step analytical approach, ensuring that your expertise shines through in a manner that is both engaging and enlightening.
        '''),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent_revise = create_openai_functions_agent(llm, tools, step_prompt)

agent_revise_executor = AgentExecutor(agent=agent_revise, tools=tools, verbose=True, reduce_k_below_max_tokens=True)

agent_revise_executor.invoke({'input':"Based on the original answer you provided about battery, I've identified several areas that need revision for a more accurate and comprehensive response:"+critique['output']+"Please answer the question based on literature search and revision.Then give five reference of your output.",'chat_history':chat_history})


In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.01)

prompt = ChatPromptTemplate.from_messages(
    [
        (   "system",
            '''
            You are recognized as a leading authority in the field of solid state battery, tasked with the pivotal role of providing expert responses to inquiries.
            Your responses must embody the highest standards of accuracy, comprehensiveness, and depth of knowledge. Approach each question with a methodical and thoughtful mindset, ensuring your answers are:

            Informative: Furnish detailed insights, drawing upon the most relevant facts and figures.
            Correct: Ensure factual accuracy in every aspect of your response.
            Knowledgeable: Display a profound understanding of the subject matter, including advanced concepts and recent advancements in the field of solid state battery.
            Holistic: Offer a well-rounded perspective, considering various facets of the topic at hand.
            When you are asked to extract information, only one set of valid information is extracted from different paper.
            As you address each question, please adhere to the following guidelines:
            Stay On Topic: Concentrate solely on the query posed. Your reply should be closely aligned with the question, avoiding tangential or unrelated content.
            Format Your Answer: Present your response in a structured manner, using either bullet points or numbered lists for clarity and ease of understanding.
            Before responding, take a moment to center yourself. Breathe deeply, and proceed with a step-by-step analytical approach, ensuring that your expertise shines through in a manner that is both engaging and enlightening.

            ''',
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

retriever_tool = create_retriever_tool(
    retriever,
    "literature_search",
    "Search for information about input questions. Make sure the information you retrieve comes from the conclusion section of a research paper!",
)

tools = [retriever_tool]

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, reduce_k_below_max_tokens=True)
user_feedback=input+thoughts+""
thought=agent_executor.invoke({"input": user_feedback, 'chat_history':[]})

with open("dialog.txt", "a", encoding="utf-8") as f:
    f.write(f"User Feedback: {user_feedback}\n")
    f.write(f"AI Summary: {thought['output']}\n")
    f.write("\n")


In [ ]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings


txt_file_path = '/content/dialog.txt'

loader = TextLoader(txt_file_path, encoding="utf-8")


ts = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=500)
documents = loader.load_and_split(text_splitter=ts)
print("Document loaded and split")


embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(documents, embedding_function)


db.save_local("faiss_index_dialog_txt")

print("FAISS index saved")
